In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('incidents.csv')
wa=df[df['state'] == 'Washington']
wa

,incident_id,date,state,city_or_county,n_killed,n_injured,latitude,longitude
45,483817,2013-03-31,Washington,Auburn,3,2,47.3149,-122.224
54,484333,2013-04-21,Washington,Federal Way,5,0,47.3030,-122.310
153,491281,2013-07-29,Washington,Granger,0,4,46.3421,-120.187
193,480071,2013-09-15,Washington,Yakima,0,6,46.5854,-120.538
323,92345,2014-01-01,Washington,Georgetown,0,1,47.5498,-122.318
...,...,...,...,...,...,...,...,...
239176,1079791,2018-03-28,Washington,Seattle,0,0,47.6138,-122.332
239399,1080903,2018-03-29,Washington,Pasco,0,1,NaN,NaN
239401,1081610,2018-03-29,Washington,Walla Walla,0,0,46.0737,-118.312
239402,1081612,2018-03-29,Washington,Yakima,0,1,46.6173,-120.512


In [3]:
import pandas as pd
import datetime
from pandarallel import pandarallel
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pandarallel.initialize(nb_workers=32)
import twint
import time
import random
import nltk
import nest_asyncio
nest_asyncio.apply()
# __import__('IPython').embed()

nltk.download('vader_lexicon')

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Python 3.6.10 (default, Apr 19 2020, 18:31:46) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.16.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: quit()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/yeraf/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
radius = 2
over = 14
def get_tweet_loc(params):
    for _ in range(10):
        try:
            c = twint.Config()
            c.Search = params['keyword']
            c.Limit = 100
            c.Geo = f"{str(params['lat'])},{str(params['lng'])},{str(params['distance'])}{params['unit']}"
            c.Pandas = True
            c.Since = params['since']
            c.Until = params['until']
            twint.run.Search(c)
            sid = SentimentIntensityAnalyzer()
            stat = {"pos": 0, "neg": 0, "neu": 0, "comp": 0, "scrape": True}
            print(twint.storage.panda.Tweets_df.shape)
            if twint.storage.panda.Tweets_df.shape[0] > 0:
                tweets = twint.storage.panda.Tweets_df['tweet']
                for tweet in tweets:
                    senvalue = sid.polarity_scores(tweet)
                    stat['pos'] += senvalue["pos"]
                    stat['neg'] += senvalue["neg"]
                    stat['neu'] += senvalue["neu"]
                    stat['comp'] += senvalue["compound"]
                stat['neg'] /= tweets.shape[0]
                stat['pos'] /= tweets.shape[0]
                stat['neu'] /= tweets.shape[0]
                stat['comp'] /= tweets.shape[0] 
            return stat
        except Exception as e:
            time.sleep(random.random() * 300)
            print('error: ', e)
            continue
    return {"pos": 0, "neg": 0, "neu": 0, "comp": 0, "scrape": True}

def get_loca_polar(r):
    response = get_tweet_loc({
            "keyword": "gun",
            "lat": r['latitude'],
            "lng": r['longitude'],
            "distance": radius,
            "unit": "km",
            "since": (datetime.datetime.strptime(r['date'], '%Y-%m-%d') - datetime.timedelta(days=over)).strftime('%Y-%m-%d'),
            "until": (datetime.datetime.strptime(r['date'], '%Y-%m-%d') + datetime.timedelta(days=over)).strftime('%Y-%m-%d')
        })
    print(response)
    if response:
        r['pos'] = response['pos']
        r['neg'] = response['neg']
        r['neu'] = response['neu']
        r['comp'] = response['comp']
        r['scrape'] = response['scrape']
    else:
        r['pos'] = 0
        r['neg'] = 0
        r['neu'] = 0
        r['comp'] = 0
        return r

wa_polar = wa.parallel_apply(get_loca_polar, axis=1)
wa_polar.to_csv('wa_polar_1_14d.csv')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.


[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.481122705245085696 2014-06-24 00:12:46 +0700 <welyons14> Guns only intensify every situation.   #theview
[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.

(0, 0)
708114513581584384 2016-03-11 09:17:10 +0700 <ac_anderson> My ski bag is camo &amp; I was carrying it to checkin @ the airport &amp; a guy yells "damn she got a big gun!" Yes, say hello to my little friends.found 0 deleted tweets in this search.



[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.

479455635080499200 2014-06-19

(4, 38){'pos': 0.0865, 'neg': 0.2665, 'neu': 0.647, 'comp': -0.14425000000000002, 'scrape': True}

594401679392530432 2015-05-02 14:22:58 +0700 <JeffMancubEMD> "That's a god damn mini gun!"  http://t.co/iaiDYDPl8K872121858665046017 2017-06-06 23:03:49 +0700 <EchemGerwe> Bigger than the Beatles Bigger than breast implants Bigger than guns, bigger than cigarettes
[!] No more data! Scraping will stop now.
[!] No more data! Scraping will stop now.{'pos': 0.021, 'neg': 0.24775000000000003, 'neu': 0.7315, 'comp': -0.407825, 'scrape': True}


593990431018356737 2015-05-01 11:08:49 +0700 <mrproam> @SenTomCotton you don't know shit..how dare you...I'll debate you on any subject except guns,ammo and killing..i know little about them
found 0 deleted tweets in this search.871859305640632321 2017-06-06 05:40:31 +0700 <Torbiesmom> Lone Wolf? Mental Health Issues? Another example of why white men with guns are the biggest threat to our safety? #OrlandoShooting  https://t.co/m5v14ylKg4(2, 38)
59387216

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57674)>, ('104.244.42.66', 443))
handle: <Handle BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57674)>, ('104.244.42.66', 443))>
Traceback (most recent call last):
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/selector_events.py", line 476, in _sock_connect_cb
    fut.set_result(None)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/futures.py", line 295, in set_result
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=None>

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57702)>, ('104.244.42.66', 443))
handle: <Handle BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57702)>, ('104.244.42.66', 443))>
Traceback (most recent call last):
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/selector_events.py", line 476, in _sock_connect_cb
    fut.set_result(None)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/futures.py", line 295, in set_result
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=None>

found 0 deleted tweets in this search.

...

found 0 deleted tweets in this search.442102394877267968 2014-03-08 07:59:59 +0700 <amandaaziegler> Who is crafty and wants to help me make outfits for EDC? Also who has a hot glue gun they wanna let me borrow?552587490297540611 2015-01-07 05:08:18 +0700 <samuraidel> @TayyibAli hahaha damn you actually replied, idk man these gun selfies are pretty corny lol...

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 58098)>, ('104.244.42.66', 443))
handle: <Handle BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 58098)>, ('104.244.42.66', 443))>
Traceback (most recent call last):
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/selector_events.py", line 476, in _sock_connect_cb
    fut.set_result(None)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/futures.py", line 295, in set_result
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=None>


760989611929632768 2016-08-04 07:03:56 +0700 <nick__nest> Taking our guns and our gorillas #ThanksObama  https://t.co/492VDAnHy1




(3, 38)[!] No more data! Scraping will stop now.{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0, 'scrape': True}
633531002707673088 2015-08-18 13:48:56 +0700 <kevinquimosing> #rangeday #rangetherapy #smithandwesson #shield #9mm #concealedcarry #ccw #guns #grouptherapy #2a…  https://t.co/sjAJEFQBkH[!] No more data! Scraping will stop now.760828370049380352 2016-08-03 20:23:13 +0700 <ir8h8fahq> @Q13FOX you should add to your report, opioid deaths per year 44,000 which is a bigger problem then guns according to the CDC.
[!] No more data! Scraping will stop now.{'pos': 0.25533333333333336, 'neg': 0.20466666666666666, 'neu': 0.5399999999999999, 'comp': 0.21036666666666667, 'scrape': True}

(0, 0)
(2, 38)

found 0 deleted tweets in this search.found 0 deleted tweets in this search.

[!] No more data! Scraping will stop now.630486232859078656 2015-08-10 04:10:06 +0700 <rrjosee> Matando el aburrimiento un ratito @ Bullseye Gun Range  https://t.co/Qhcj3u3DwK{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0,

756361060555325440 2016-07-22 12:31:44 +0700 <yunkin_donuts> Love is patient  Love is kind Love is also a machine gun That can cut you down to size @…  https://t.co/sa4PjGLQgJ

755049522955333633 2016-07-18 21:40:09 +0700 <SeattleCP> Experts at West Coast Armory North showing us the latest technology available for safe gun storage at home.…  https://t.co/IIKsbBreYKfound 0 deleted tweets in this search.

754322755420495872 2016-07-16 21:32:14 +0700 <SeattleCP> New 9/11 document reveals no smoking gun of Saudi complicity  https://t.co/qSvOWCfK1u
753754229865525248 2016-07-15 07:53:07 +0700 <SeattleCP> Deputy fires "1 in a billion" shot into suspect's gun barrel  https://t.co/XcZE7QOQwm
(2, 38)
{'pos': 0.20450000000000002, 'neg': 0.18, 'neu': 0.6154999999999999, 'comp': 0.07600000000000001, 'scrape': True}
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(7, 38)
{'pos': 0.05242857142857143, 'neg': 0.15157142857142855, 'neu': 0.796, 'comp': -0.17772857142857